# Data Setting

# Stratify folds considering bbox_counts and unique class distributions

In [83]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, GroupKFold
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import warnings

warnings.filterwarnings('ignore')
n_folds = 5
df = pd.read_csv(r'df_annotate.csv')
df_backup = df.copy()
#df = pd.DataFrame(df)
df = df[df['class_id'] != 0]
print(df.shape)
df_meta = pd.read_csv(r'df_meta.csv')
df_meta = df_meta.rename(columns={"image":"image_id"})
df = df.merge(df_meta[['image_id','study']], on='image_id', how='left')
df = df.rename(columns={"study":"study_id"})
fold_info = pd.read_csv(r'df_fold_rand830.csv')
df = df.merge(fold_info, on='study_id', how='left')
print(df.shape)
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=101)

fold = 0
IMAGE_SIZE = 512

df = df.reset_index(drop=True)

(7853, 12)
(7853, 14)


In [84]:
df['class_id'] = 0

In [85]:
df['ncx'] = df['nx'] + df['nw'] /2
df['ncy'] = df['ny'] + df['nh'] / 2

In [86]:
df

,image_id,width,height,class_id,x,y,w,h,nx,ny,nw,nh,study_id,fold,ncx,ncy
0,000a312787f2,4256,3488,0,789.28836,582.43035,1026.65662,1917.30292,0.185453,0.166981,0.241226,0.549685,5776db0cec75,2,0.306066,0.441824
1,000a312787f2,4256,3488,0,2245.91208,591.20528,1094.66162,1761.54944,0.527705,0.169497,0.257204,0.505031,5776db0cec75,2,0.656307,0.422013
2,0012ff7358bc,3056,2544,0,677.42216,197.97662,867.79767,999.78214,0.221670,0.077821,0.283965,0.392996,9d514ce429a7,1,0.363652,0.274319
3,0012ff7358bc,3056,2544,0,1792.69064,402.55250,617.02734,1204.35800,0.586613,0.158236,0.201907,0.473411,9d514ce429a7,1,0.687567,0.394942
4,001398f4ff4f,4280,3520,0,2729.00000,2181.33331,948.00012,604.00000,0.637617,0.619697,0.221495,0.171591,28dddc8559b2,0,0.748364,0.705492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7848,ffd9b6cf2961,3050,2388,0,707.25199,722.07926,392.14044,849.18683,0.231886,0.302378,0.128571,0.355606,7eed9af03814,2,0.296171,0.480181
7849,ffdc682f7680,4256,3488,0,2729.27083,332.26044,1496.25016,2604.58334,0.641276,0.095258,0.351563,0.746727,a0cb0b96fb3d,4,0.817057,0.468622
7850,ffdc682f7680,4256,3488,0,1005.81250,1584.67711,662.22913,775.83337,0.236328,0.454323,0.155599,0.222429,a0cb0b96fb3d,4,0.314128,0.565537
7851,ffe942c8655f,1387,1140,0,208.86463,91.53448,450.96747,628.05473,0.150587,0.080293,0.325139,0.550925,7d82d53204b8,3,0.313157,0.355756


In [87]:
df_train

,image_id,width,height,class_id,x,y,w,h,nx,ny,nw,nh,study_id,fold,ncx,ncy
0,000a312787f2,4256,3488,0,789.28836,582.43035,1026.65662,1917.30292,0.185453,0.166981,0.241226,0.549685,5776db0cec75,2,0.306066,0.441824
1,000a312787f2,4256,3488,0,2245.91208,591.20528,1094.66162,1761.54944,0.527705,0.169497,0.257204,0.505031,5776db0cec75,2,0.656307,0.422013
2,0012ff7358bc,3056,2544,0,677.42216,197.97662,867.79767,999.78214,0.221670,0.077821,0.283965,0.392996,9d514ce429a7,1,0.363652,0.274319
3,0012ff7358bc,3056,2544,0,1792.69064,402.55250,617.02734,1204.35800,0.586613,0.158236,0.201907,0.473411,9d514ce429a7,1,0.687567,0.394942
4,001bd15d1891,3408,2800,0,623.23328,1050.00000,714.00000,1106.00000,0.182874,0.375000,0.209507,0.395000,dfd9fdd85a3e,4,0.287627,0.572500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6281,ffd9b6cf2961,3050,2388,0,707.25199,722.07926,392.14044,849.18683,0.231886,0.302378,0.128571,0.355606,7eed9af03814,2,0.296171,0.480181
6282,ffdc682f7680,4256,3488,0,2729.27083,332.26044,1496.25016,2604.58334,0.641276,0.095258,0.351563,0.746727,a0cb0b96fb3d,4,0.817057,0.468622
6283,ffdc682f7680,4256,3488,0,1005.81250,1584.67711,662.22913,775.83337,0.236328,0.454323,0.155599,0.222429,a0cb0b96fb3d,4,0.314128,0.565537
6284,ffe942c8655f,1387,1140,0,208.86463,91.53448,450.96747,628.05473,0.150587,0.080293,0.325139,0.550925,7d82d53204b8,3,0.313157,0.355756


In [88]:

df_train = df[df['fold']!=fold]
df_valid = df[df['fold']==fold]
df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)

## Train data generation

In [89]:
fold

0

In [93]:
import os
import shutil
from tqdm import tqdm
if True:
    for fold in [fold]:
        #df_train = pd.merge(df, df_folds[df_folds['fold']!=fold], on='image_id')
        for name,mini in tqdm(df_train.groupby('image_id')):
            path2save = 'train2017/'
            if not os.path.exists('convertor/fold_{}/labels/'.format(fold)+path2save):
                os.makedirs('convertor/fold_{}/labels/'.format(fold)+path2save)
            with open('convertor/fold_{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                row = mini[['class_id','ncx','ncy','nw','nh']].astype(float).values
                #row['class_id'] = row['class_id'].astype(int)
                row = row.astype(str)
                for j in range(len(row)):
                    #if j == 0:
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('convertor/fold_{}/images/{}'.format(fold,path2save)):
                os.makedirs('convertor/fold_{}/images/{}'.format(fold,path2save))
            shutil.copy("train/{}.png".format(name),'convertor/fold_{}/images/{}/{}.png'.format(fold,path2save,name))

100%|█████████████████████████████████████████████████████████████████████████████| 3436/3436 [00:06<00:00, 492.92it/s]


## Validation Data Generation

In [92]:
import shutil
from tqdm import tqdm
if True:
    for fold in [fold]:
        #df_valid = pd.merge(df, df_folds[df_folds['fold']==fold], on='image_id')
        for name,mini in tqdm(df_valid.groupby('image_id')):
            path2save = 'valid2017/'
            if not os.path.exists('convertor/fold_{}/labels/'.format(fold)+path2save):
                os.makedirs('convertor/fold_{}/labels/'.format(fold)+path2save)
            with open('convertor/fold_{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                row = mini[['class_id','ncx','ncy','nw','nh']].astype(float).values
                #row['class_id'] = row['class_id'].astype(int)
                row = row.astype(str)
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('convertor/fold_{}/images/{}'.format(fold,path2save)):
                os.makedirs('convertor/fold_{}/images/{}'.format(fold,path2save))
            shutil.copy("train/{}.png".format(name),'convertor/fold_{}/images/{}/{}.png'.format(fold,path2save,name))

100%|███████████████████████████████████████████████████████████████████████████████| 858/858 [00:01<00:00, 506.78it/s]


In [71]:
df_valid

,image_id,width,height,class_id,x,y,w,h,nx,ny,nw,nh,study_id,fold,ncx,ncy
0,001398f4ff4f,4280,3520,0,2729.00000,2181.33331,948.00012,604.00000,0.637617,0.619697,0.221495,0.171591,28dddc8559b2,0,0.748364,0.705492
1,0022227f5adf,3050,2539,0,1857.20650,508.30565,376.02734,399.52911,0.608920,0.200199,0.123288,0.157357,84543edc24c2,0,0.670564,0.278878
2,0133bf9a5f5c,2836,2336,0,2147.62756,552.85333,580.10667,638.50667,0.757273,0.236667,0.204551,0.273333,5be2cf346b1e,0,0.859549,0.373333
3,0133bf9a5f5c,2836,2336,0,566.93423,1027.84000,654.08000,572.32000,0.199906,0.440000,0.230635,0.245000,5be2cf346b1e,0,0.315224,0.562500
4,01376c1ba556,4248,3480,0,696.48983,1070.04082,1060.57147,1297.30615,0.163957,0.307483,0.249664,0.372789,3c88bd82fb37,0,0.288789,0.493878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,fec9646c8c88,3032,2436,0,2102.26795,1729.56000,523.74000,661.78000,0.693360,0.710000,0.172737,0.271667,da0d817b69fe,0,0.779729,0.845833
1563,fee4447733fa,2431,2010,0,1541.01726,353.83928,327.09473,896.48175,0.633903,0.176039,0.134552,0.446011,baf7f4d4a312,0,0.701178,0.399045
1564,fee4447733fa,2431,2010,0,214.46658,442.67985,357.38124,781.39282,0.088222,0.220239,0.147010,0.388753,baf7f4d4a312,0,0.161727,0.414615
1565,fef3c9e51d8d,4248,3480,0,904.35938,341.97660,1026.04688,1877.85938,0.212891,0.098269,0.241536,0.539615,990fc38a16ae,0,0.333659,0.368077


# Make yaml file

In [94]:
from os import listdir
from os.path import isfile, join
import yaml
import glob

#cwd = './ '
for fold in [fold]:
    with open(f'train{fold}.txt', 'w') as f:

        for path in glob.glob(f'convertor/fold_{fold}/images/train2017/*png'):
            print(path)
            f.write(path+'\n')
            
    with open(f'val{fold}.txt', 'w') as f:
        for path in glob.glob(f'convertor/fold_{fold}/images/valid2017/*png'):
            f.write(path+'\n')

data = dict(
    train =  f'train{fold}.txt',
    val   =  f'val{fold}.txt',
    nc    = 1,
    names = ['0. opacity']
    )

with open(join('covid19.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(join('covid19.yaml'), 'r')
print('\nyaml:')
print(f.read())

convertor/fold_0/images/train2017\000a312787f2.png
convertor/fold_0/images/train2017\0012ff7358bc.png
convertor/fold_0/images/train2017\001bd15d1891.png
convertor/fold_0/images/train2017\002e9b2128d0.png
convertor/fold_0/images/train2017\00326161e51e.png
convertor/fold_0/images/train2017\0044e449aae1.png
convertor/fold_0/images/train2017\0049814626c8.png
convertor/fold_0/images/train2017\004cbd797cd1.png
convertor/fold_0/images/train2017\0060e4f54aa8.png
convertor/fold_0/images/train2017\006e2ac85d57.png
convertor/fold_0/images/train2017\007b2567d83e.png
convertor/fold_0/images/train2017\007cf31356c6.png
convertor/fold_0/images/train2017\008ca392cff3.png
convertor/fold_0/images/train2017\008ff73b4ef3.png
convertor/fold_0/images/train2017\00a129830f4e.png
convertor/fold_0/images/train2017\00b0891276a3.png
convertor/fold_0/images/train2017\00b767b49fcd.png
convertor/fold_0/images/train2017\00b8180bd3a8.png
convertor/fold_0/images/train2017\00e37a390f0f.png
convertor/fold_0/images/train20


yaml:
names:
- 0. opacity
nc: 1
train: train0.txt
val: val0.txt



In [95]:
print(fold)

0


In [96]:
import gc
gc.collect()

13874

In [98]:
# With WBF 0.6
!python train.py --img {IMAGE_SIZE} --batch 4 --epochs 50 --data covid19.yaml --weights yolov5l.pt --hyp data/hyps/covidhyp.finetune.yaml --single-cls 

^C


# Inference or Evaluation

In [ ]:
!cp gdrive/My\ Drive/Colab\ Notebooks/vin_save/best_fold1_imsize[1024,\ 1024].pt .
!cp gdrive/My\ Drive/Colab\ Notebooks/vin_save/best_fold2_imsize[1024,\ 1024].pt .
!cp gdrive/My\ Drive/Colab\ Notebooks/vin_save/best_fold3_imsize[1024,\ 1024].pt .
!cp gdrive/My\ Drive/Colab\ Notebooks/vin_save/best_fold4_imsize[1024,\ 1024].pt .
!cp gdrive/My\ Drive/Colab\ Notebooks/vin_save/best_fold0_imsize[1024,\ 1024].pt .

In [ ]:
from google.colab import files
files.upload()

Saving test.csv to test.csv


{'test.csv': b'image_id,width,height\n83caa8a85e03606cf57e49147d7ac569,2304,2880\n7550347fa2bb96c2354a3716dfa3a69c,2538,3095\n74b23792db329cff5843e36efb8aa65a,2788,3120\n94568a546be103177cb582d3e91cd2d8,1994,2430\n6da36354fc904b63bc03eb3884e0c35c,2056,2376\n5b3620a56668010877cfafa9322e18b5,2336,2788\n5157d5c9b1f0004b29310652ea54e825,2517,3028\n74897408dcbd69ef7c76cbbbbeb722d6,3072,3072\n271290de7e7e4fb8aa1c31327f48f4b5,2067,2492\ncdb9c0046cc8f9e32fcafc5517000525,3072,3072\neb4c1f7a8fcd7ee076ae9f5eca8d05fc,1994,2430\n366fb4e84d6940ea74267b32edbd1311,3072,3072\nf5f77203f00504c40b45a65d23cd8948,2504,2930\n212dc39b6ba6941b034c2cd92cf97a8d,3072,3072\na5eabc3dd2734b211c9b09205443a2ea,2336,2836\nc374e62c67f8e7a4d460e37d56bfa852,2231,2398\ncb3b210015a31adab114493e1150eaf8,1994,2430\n6cbf4295b5b72bc01ef6fd171ef7733e,2276,2446\n564141a415cfa32841bb2de146faa5eb,3000,3000\nfc28fc9ede2c505c2125343c048124b6,2336,2472\n2f775125346cd62232d8e835794941c0,2321,2742\n1fae839a73abec3f3d7436fa2bce4630,2048,

In [ ]:
test_df = pd.read_csv('vin/test.csv')
#test_df = df_valid.copy()
IMAGE_SIZE = 1024

In [ ]:
test_df.head()

,image_id,width,height
0,83caa8a85e03606cf57e49147d7ac569,2304,2880
1,7550347fa2bb96c2354a3716dfa3a69c,2538,3095
2,74b23792db329cff5843e36efb8aa65a,2788,3120
3,94568a546be103177cb582d3e91cd2d8,1994,2430
4,6da36354fc904b63bc03eb3884e0c35c,2056,2376


- A check if weight exists

In [ ]:
import glob
def deletetext():
  for i in range(5):
    for x in glob.glob(f'runs{fold}/detect/exp/labels/*txt'):
        os.remove(x)
deletetext()

In [ ]:
import gc
gc.collect()

212

In [ ]:
!python detect.py --weights best_fold0_imsize[1024,\ 1024].pt --img 768 --conf 0.001 --iou 0.4 --source convertor/fold_0/images/valid2017 --save-txt --save-conf --exist-ok --project runs0/detect/
!python detect.py --weights best_fold1_imsize[1024,\ 1024].pt --img 768 --conf 0.001 --iou 0.4 --source convertor/fold_1/images/valid2017 --save-txt --save-conf --exist-ok --project runs1/detect/ 
!python detect.py --weights best_fold2_imsize[1024,\ 1024].pt --img 768 --conf 0.001 --iou 0.4 --source convertor/fold_2/images/valid2017 --save-txt --save-conf --exist-ok --project runs2/detect/ 
!python detect.py --weights best_fold3_imsize[1024,\ 1024].pt --img 768 --conf 0.001 --iou 0.4 --source convertor/fold_3/images/valid2017 --save-txt --save-conf --exist-ok --project runs3/detect/ 
!python detect.py --weights best_fold4_imsize[1024,\ 1024].pt --img 768 --conf 0.001 --iou 0.4 --source convertor/fold_4/images/valid2017 --save-txt --save-conf --exist-ok --project runs4/detect/ 

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.001, device='', exist_ok=True, img_size=768, iou_thres=0.4, name='exp', project='runs0/detect/', save_conf=True, save_txt=True, source='convertor/fold_0/images/valid2017', update=False, view_img=False, weights=['best_fold0_imsize[1024, 1024].pt'])
YOLOv5 🚀  torch 1.8.1+cu101 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

Fusing layers... 
Model Summary: 606 layers, 141121108 parameters, 0 gradients, 221.5 GFLOPS
image 1/879 /content/convertor/fold_0/images/valid2017/0061cf6d35e253b6e7f03940592cc35e.png: 768x768 1 Aortic enlargement, 1 Atelectasis, 5 Calcifications, 1 Lung Opacity, 4 Nodule/Masss, 5 Other lesions, 6 Pleural thickenings, 5 Pulmonary fibrosiss, Done. (0.086s)
image 2/879 /content/convertor/fold_0/images/valid2017/00675cd546313f912cadd4ad54415d69.png: 768x768 3 Aortic enlargements, 3 Calcifications, 3 Cardiomegalys, 1 Lung Opacity, 1 Nodule/Mass, 5 Other lesions, 7 Pleural thickenings, 2 Pulmonary fibrosi

In [ ]:
import gc
gc.collect()

380

# Inference Plot

In [ ]:
def yolo2voc(image_height, image_width, bboxes):
    bboxes = bboxes.copy().astype(float)
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]] * image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]] * image_height

    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - (bboxes[..., [2, 3]] / 2)
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

    return bboxes

In [ ]:
import glob
image_ids = []
PredictionStrings = []

for file_path in tqdm(glob.glob('runs2/detect/exp/labels/*txt')):
    image_id = file_path.split('/')[-1].split('.')[0]
    w, h = test_df.loc[test_df.image_id==image_id,['width', 'height']].values[0]
    f = open(file_path, 'r')
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
    bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 1).astype(str))
    for idx in range(len(bboxes)):
        bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
    image_ids.append(image_id)
    PredictionStrings.append(' '.join(bboxes))

100%|██████████| 3000/3000 [00:06<00:00, 460.14it/s]


In [ ]:
test_df

,image_id,width,height
0,83caa8a85e03606cf57e49147d7ac569,2304,2880
1,7550347fa2bb96c2354a3716dfa3a69c,2538,3095
2,74b23792db329cff5843e36efb8aa65a,2788,3120
3,94568a546be103177cb582d3e91cd2d8,1994,2430
4,6da36354fc904b63bc03eb3884e0c35c,2056,2376
...,...,...,...
2995,7f5503caa936a623b4388fbd88e890c5,2642,3170
2996,c97e54a78bab9c05ce2e04fe6c284bcd,3072,3072
2997,33218cf183c1224a74ccfb514e827e15,2336,2836
2998,04b700c4815f088728db9f093c739707,2484,3064


# For NMS IOU THRESHOLD AT 0.5

In [ ]:
def yolo2xyxy(image_height, image_width, bboxes):
    bboxes = bboxes.copy().astype(float)
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]] * image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]] * image_height

    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - (bboxes[..., [2, 3]] / 2)
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

    return bboxes

In [ ]:
import glob
image_ids = []
PredictionStrings = []
total_concat = pd.DataFrame({'image_id':[], 'class_id':[], 'pred':[], 'x_min':[], 'y_min':[], 'x_max':[], 'y_max':[]})
for fold in range(5):

  for i, file_path in enumerate(glob.glob(f'runs{fold}/detect/exp/labels/*txt')):
    image_id = file_path.split('/')[-1].split('.')[0]
    w, h = test_df.loc[test_df.image_id==image_id,['width', 'height']].values[0]
    f = open(file_path, 'r')
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
    data = np.concatenate((data[:, :2], np.round(yolo2xyxy(IMAGE_SIZE, IMAGE_SIZE, data[:, 2:]))), axis =1)
    temp_concat = pd.DataFrame({'image_id':[image_id]*len(data), 'class_id':data[:,0], 'pred':data[:,1], 'x_min':data[:,2], 'y_min':data[:,3], 'x_max':data[:,4], 'y_max':data[:,5]})
    total_concat = pd.concat([total_concat, temp_concat], axis=0)
    
print(total_concat)
    #data[:,2:] = data[:,2:]
    #bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 1).astype(str))
    #for idx in range(len(bboxes)):
    #    bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
    #image_ids.append(image_id)
    #PredictionStrings.append(' '.join(bboxes))

                            image_id  class_id      pred  ...  y_min   x_max  y_max
0   12202997d3fc0d506d2514a4c1995d15       7.0  0.001103  ...  344.0   757.0  461.0
1   12202997d3fc0d506d2514a4c1995d15       8.0  0.001352  ...  208.0  1000.0  808.0
2   12202997d3fc0d506d2514a4c1995d15       6.0  0.001379  ...  208.0  1000.0  808.0
3   12202997d3fc0d506d2514a4c1995d15       5.0  0.001502  ...  208.0  1000.0  808.0
4   12202997d3fc0d506d2514a4c1995d15      11.0  0.002132  ...  201.0   441.0  226.0
..                               ...       ...       ...  ...    ...     ...    ...
6   4d2071baccfd994701c3a5ceb3f64f6f      11.0  0.010170  ...   94.0   742.0  139.0
7   4d2071baccfd994701c3a5ceb3f64f6f      11.0  0.030655  ...  104.0   399.0  130.0
8   4d2071baccfd994701c3a5ceb3f64f6f      11.0  0.032867  ...  104.0   336.0  149.0
9   4d2071baccfd994701c3a5ceb3f64f6f      11.0  0.035614  ...   96.0   714.0  123.0
10  4d2071baccfd994701c3a5ceb3f64f6f       3.0  0.758789  ...  450.0   814.0

In [ ]:
total_concat['x_max'].max()

1024.0

# Ensemble NMS

In [ ]:
!pip install ensemble_boxes
from ensemble_boxes import *
import warnings
from collections import Counter
from tqdm import tqdm

from path import Path
test_imagepaths = ("test/" + total_concat['image_id'] + ".png").unique()
print("Number of Images with abnormalities:",len(test_imagepaths))
anno_count = df_for_plot.shape[0]
print("Number of Annotations with abnormalities:", anno_count)
Path("train/" + imagepaths[0] + ".png").stem

iou_thr = 0.4
skip_box_thr = 0.0001
viz_images = []
paths = []
total_boxes = []
total_labels = []
total_scores = []
for i, path in tqdm(enumerate(test_imagepaths)):

    #img_array  = cv2.imread(path)
    image_basename = Path(path).stem
    #print(f"(\'{image_basename}\', \'{path}\')")
    img_annotations = total_concat[total_concat.image_id==image_basename]

    boxes_viz = img_annotations[['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().tolist()
    labels_viz = img_annotations['class_id'].to_numpy().tolist()
    
    print("Bboxes before nms:\n", len(boxes_viz))
    #print("Labels before nms:\n", labels_viz)
    
    ## Visualize Original Bboxes
    #img_before = img_array.copy()
    for box, label in zip(boxes_viz, labels_viz):
        x_min, y_min, x_max, y_max = (box[0], box[1], box[2], box[3])
        color = label2color[int(label)]
        #img_before = draw_bbox(img_before, list(np.int_(box)), viz_labels[label], color)
    #viz_images.append(img_before)
    
    boxes_list = []
    scores_list = []
    labels_list = []
    weights = []
    
    boxes_single = []
    labels_single = []
    scores_single = []

    cls_ids = img_annotations['class_id'].unique().tolist()
    count_dict = Counter(img_annotations['class_id'].tolist())
    #print(count_dict)

    for cid in cls_ids:       
        ## Performing Fusing operation only for multiple bboxes with the same label
        if count_dict[cid]==1:
            labels_single.append(cid)
            boxes_single.append(img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().squeeze().tolist())
            scores_single.append(img_annotations[img_annotations.class_id==cid][['pred']].to_numpy().squeeze().tolist())

        else:
            cls_list =img_annotations[img_annotations.class_id==cid]['class_id'].tolist()
            labels_list.append(cls_list)
            bbox = img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()
            ## Normalizing Bbox by Image Width and Height
            bbox = bbox/(IMAGE_SIZE, IMAGE_SIZE, IMAGE_SIZE, IMAGE_SIZE)
            bbox = np.clip(bbox, 0, 1)
            score = img_annotations[img_annotations.class_id==cid][['pred']].to_numpy().squeeze().tolist()
            #print(scores)
            #print(bbox, scores)
            boxes_list.append(bbox.tolist())
            scores_list.append(score)

            weights.append(1)
            
    # Perform NMS
    #print(boxes)
    if len(boxes_list) > 0:
        #print("boxes", boxes, "scores",scores, 'box_labels',box_labels)
        #print("before", labels_list)
        #print(scores_list)
        #print(boxes_list)
        boxes, scores, box_labels = nms(boxes_list, scores_list, labels_list, weights=None, iou_thr=iou_thr)
        #print("after", box_labels, scores, boxes)
        #print(len(boxes), len(scores))
        boxes = boxes*(IMAGE_SIZE, IMAGE_SIZE, IMAGE_SIZE, IMAGE_SIZE)
        boxes = boxes.round(1).tolist()
        box_labels = box_labels.astype(int).tolist()
        scores = scores.tolist()

    boxes.extend(boxes_single)

    box_labels.extend(labels_single)
    scores.extend(scores_single)
    
    paths.extend([path]*len(boxes))
    total_boxes.extend(boxes)
    total_labels.extend(box_labels)
    total_scores.extend(scores)
    print("Bboxes after nms:\n", len(boxes))

1it [00:00,  8.46it/s]

Number of Images with abnormalities: 3000
Number of Annotations with abnormalities: 36096
Bboxes before nms:
 362
Bboxes after nms:
 75
Bboxes before nms:
 617


3it [00:00,  8.79it/s]

Bboxes after nms:
 133
Bboxes before nms:
 233
Bboxes after nms:
 56
Bboxes before nms:
 254


5it [00:00,  8.59it/s]

...

Bboxes after nms:
 59
Bboxes before nms:
 535
Bboxes after nms:
 153
Bboxes before nms:
 897


2999it [05:31,  8.63it/s]

Bboxes after nms:
 202
Bboxes before nms:
 172
Bboxes after nms:
 57
Bboxes before nms:
 113


3000it [05:31,  9.04it/s]

Bboxes after nms:
 31


In [ ]:
width_test_dict = {x:y for x, y in zip(test_df.image_id.tolist(), test_df.width.tolist())}
height_test_dict = {x:y for x, y in zip(test_df.image_id.tolist(), test_df.height.tolist())}
total_df = pd.DataFrame({'image_id':map(lambda x: x.split("/")[-1].split(".")[0], paths), 'class_id':total_labels, 'pred':total_scores})

df_coords = pd.DataFrame(total_boxes, columns=['x_min', 'y_min', 'x_max', 'y_max'])
df = pd.concat([total_df, df_coords], axis=1)

df['x_min'] = df.apply(lambda x: x['x_min']*width_test_dict[x['image_id']]/IMAGE_SIZE, axis=1).astype(int)
df['x_max'] = df.apply(lambda x: x['x_max']*width_test_dict[x['image_id']]/IMAGE_SIZE, axis=1).astype(int)
df['y_min'] = df.apply(lambda x: x['y_min']*height_test_dict[x['image_id']]/IMAGE_SIZE, axis=1).astype(int)
df['y_max'] = df.apply(lambda x: x['y_max']*height_test_dict[x['image_id']]/IMAGE_SIZE, axis=1).astype(int)

df.iloc[:,1] = df.iloc[:,1].astype(int)
#df.iloc[:,2] = df.iloc[:,2].astype(int)
df.iloc[:,3:] = df.iloc[:,3:].astype(int)


In [ ]:
new_sub_df = pd.DataFrame({'image_id':[], 'PredictionString': []})
for i, image_id in tqdm(enumerate(df['image_id'].unique())):
    image_id_df = df.loc[df.image_id == image_id]
    image_id_df.pred = image_id_df['pred'].map(lambda x: round(x, 1))
    #print(image_id_df.class_id)
    image_id_df = image_id_df.astype(str)
    image_id_df['string'] = image_id_df.apply(lambda x: (x['class_id']+ " " +x['pred'] + " " + x['x_min'] + " " + x['y_min'] + " " + x['x_max'] + " " + x['y_max']), axis=1)
    #print(image_id_df['string'])
    total_string = " ".join(image_id_df['string'].tolist())
    #print(total_string)
    temp_df = pd.DataFrame({'image_id':[image_id], 'PredictionString':[total_string]})
    #print(temp_df)
    new_sub_df = pd.concat([new_sub_df, temp_df], axis=0)

3000it [01:06, 45.29it/s]


In [ ]:
new_sub_df = new_sub_df.reset_index(drop=True)
new_sub_df.head()

,image_id,PredictionString
0,12202997d3fc0d506d2514a4c1995d15,0 0.4 983 766 1236 1029 0 0.0 1109 514 1892 18...
1,c2c1843b4ca2644b92d5387358dc8bc2,0 0.3 1182 837 1832 1283 0 0.1 1112 752 2119 2...
2,c28e30456f3d1b1dda8a22f958b7aa96,0 0.7 961 616 1148 830 0 0.0 597 0 1314 147 0 ...
3,2b7ee30a6af65e9d2915764004122b36,0 0.2 1111 723 1292 913 1 0.0 1540 1105 1795 1...
4,3019aec706bd013e1e3348564fbfd086,0 0.7 1154 778 1384 1086 0 0.0 1335 721 1676 1...


#Ensemble Prediction

In [ ]:
#pred_df = pd.DataFrame({'image_id':image_ids, 'PredictionString':PredictionStrings})
sub_df = pd.merge(test_df, new_sub_df, on='image_id', how='left').fillna('14 1 0 0 1 1')
sub_df = sub_df[['image_id', 'PredictionString']]
sub_df.to_csv("(5ensembleaugment)submission.csv", index=False)
from google.colab import files
#files.download("(5ensemble)submission.csv")
sub_df.tail()


,image_id,PredictionString
2995,7f5503caa936a623b4388fbd88e890c5,0 0.6 1375 795 1640 1068 3 0.4 988 1622 2071 2...
2996,c97e54a78bab9c05ce2e04fe6c284bcd,0 0.8 1704 978 2070 1407 0 0.0 1368 1686 2418 ...
2997,33218cf183c1224a74ccfb514e827e15,0 0.0 1304 775 1610 1129 0 0.0 894 1334 1884 1...
2998,04b700c4815f088728db9f093c739707,0 0.6 1309 1068 1576 1361 0 0.0 1341 1065 1508...
2999,14da9051525bd2504dd56938f92644ef,0 0.4 987 806 1195 1029 0 0.0 804 811 1193 122...


# No Ensemble Prediction

In [ ]:
pred_df = pd.DataFrame({'image_id':image_ids, 'PredictionString':PredictionStrings})
sub_df = pd.merge(test_df, pred_df, on='image_id', how='left').fillna('14 1 0 0 1 1')
sub_df = sub_df[['image_id', 'PredictionString']]
sub_df.to_csv("submission.csv", index=False)
sub_df.tail()

,image_id,PredictionString
2995,7f5503caa936a623b4388fbd88e890c5,10 0.0 235 1938 323 2148 5 0.0 1932 1573 2307 ...
2996,c97e54a78bab9c05ce2e04fe6c284bcd,8 0.0 1704 978 2064 1410 11 0.0 1824 549 2292 ...
2997,33218cf183c1224a74ccfb514e827e15,7 0.0 1722 1365 2058 1872 5 0.0 1455 1005 2030...
2998,04b700c4815f088728db9f093c739707,13 0.0 252 1987 1009 2501 9 0.0 822 1676 1046 ...
2999,14da9051525bd2504dd56938f92644ef,11 0.0 584 522 709 560 3 0.0 989 812 1200 1016...


In [ ]:
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#If Two Classifier

In [ ]:
#!cp gdrive/My\ Drive/Colab\ Notebooks/vin_2classifier/\(vinbig2class\)1024B5NoisyDropjustsubmission.csv .
!cp gdrive/My\ Drive/Colab\ Notebooks/vin_2classifier/(vinbig2class)(ensemble)NoisyDropjustsubmission.csv .

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `cp gdrive/My\ Drive/Colab\ Notebooks/vin_2classifier/(vinbig2class)(ensemble)NoisyDropjustsubmission.csv .'


In [ ]:
pred_2class

,image_id,target
0,002a34c58c5b758217ed1f584ccbcfe9,0.004992
1,004f33259ee4aef671c2b95d54e4be68,0.057574
2,008bdde2af2462e86fd373a445d0f4cd,0.871345
3,009bc039326338823ca3aa84381f17f1,0.061938
4,00a2145de1886cb9eb88869c85d74080,0.841936
...,...,...
2995,ff91fb82429a27521bbec8569b041f02,0.898296
2996,ff9fcc4087ed5e941209aa3fa948e364,0.921694
2997,ffaa288c8abca300974f043b57d81521,0.164820
2998,ffc441e0c8b7153844047483a577e7c3,0.318541


In [ ]:
pred_2class = pd.read_csv("gdrive/My Drive/Colab Notebooks/vin_2classifier/(vinbig2class)(ensemble)NoisyDropjustsubmission.csv", index_col=0)
pred_2class.columns =['image_id', 'target']
low_threshold = 0.0
high_threshold = 0.95
pred_2class

NORMAL = "14 1 0 0 1 1"

pred_det_df = pd.read_csv("(5ensembleaugment)submission.csv")
n_normal_before = len(pred_det_df.query("PredictionString == @NORMAL"))
merged_df = pd.merge(pred_det_df, pred_2class, on="image_id", how="left")


if "target" in merged_df.columns:
    merged_df["class0"] = 1 - merged_df["target"]

c0, c1, c2 = 0, 0, 0
for i in range(len(merged_df)):
    p0 = merged_df.loc[i, "class0"]
    if p0 < low_threshold:

        c0 += 1
    elif low_threshold <= p0 and p0 < high_threshold:

        merged_df.loc[i, "PredictionString"] += f" 14 {round(p0,1)} 0 0 1 1"
        c1 += 1
    else:

        merged_df.loc[i, "PredictionString"] = NORMAL
        c2 += 1

n_normal_after = len(merged_df.query("PredictionString == @NORMAL"))
print(
    f"n_normal: {n_normal_before} -> {n_normal_after} with threshold {low_threshold} & {high_threshold}"
)
print(f"Keep {c0} Add {c1} Replace {c2}")
submission_filepath = str(f"{IMAGE_SIZE}(allfoldfold,conf0.001detectiou=0.4,nmsout=0.4)submission.csv")
submission_df = merged_df[["image_id", "PredictionString"]]
submission_df.to_csv(submission_filepath, index=False)
print(f"Saved to {submission_filepath}")

n_normal: 0 -> 1891 with threshold 0.0 & 0.95
Keep 0 Add 1109 Replace 1891
Saved to 1024(allfoldfold,conf0.001detectiou=0.4,nmsout=0.4)submission.csv


In [ ]:
pred_2class = pd.read_csv("(vinbig2class)(ensemble)NoisyDropjustsubmission.csv")
pred_2class.columns =['image_id', 'target']
low_threshold = 0.0
high_threshold = 0.95
pred_2class

NORMAL = "14 1 0 0 1 1"

pred_det_df = pd.read_csv("(5ensemble)submission.csv")
n_normal_before = len(pred_det_df.query("PredictionString == @NORMAL"))
merged_df = pd.merge(pred_det_df, pred_2class, on="image_id", how="left")


if "target" in merged_df.columns:
    merged_df["class0"] = 1 - merged_df["target"]

c0, c1, c2 = 0, 0, 0
for i in range(len(merged_df)):
    p0 = merged_df.loc[i, "class0"]
    if p0 < low_threshold:

        c0 += 1
    elif low_threshold <= p0 and p0 < high_threshold:

        merged_df.loc[i, "PredictionString"] += f" 14 {round(p0,1)} 0 0 1 1"
        c1 += 1
    else:

        merged_df.loc[i, "PredictionString"] = NORMAL
        c2 += 1

n_normal_after = len(merged_df.query("PredictionString == @NORMAL"))
print(
    f"n_normal: {n_normal_before} -> {n_normal_after} with threshold {low_threshold} & {high_threshold}"
)
print(f"Keep {c0} Add {c1} Replace {c2}")
submission_filepath = str("(2class,fold1,conf0)submission.csv")
submission_df = merged_df[["image_id", "PredictionString"]]
submission_df.to_csv(submission_filepath, index=False)
print(f"Saved to {submission_filepath}")

FileNotFoundError: ignored


image_id	PredictionString
2995	7f5503caa936a623b4388fbd88e890c5	3 0.2 986 1613 2074 2025 0 0.6 1380 799 1641 1...
2996	c97e54a78bab9c05ce2e04fe6c284bcd	11 0.2 1836 588 2235 672 3 0.3 1368 1674 2412 ...
2997	33218cf183c1224a74ccfb514e827e15	7 0.3 506 991 1001 1631 3 0.8 874 1346 1891 17...
2998	04b700c4815f088728db9f093c739707	11 0.3 703 592 1043 667 0 0.7 1305 1071 1572 1...
2999	14da9051525bd2504dd56938f92644ef	0 0.2 989 809 1198 1042

In [ ]:
from google.colab import files
files.download(f'{IMAGE_SIZE}(allfoldfold,conf0.001detectiou=0.4,nmsout=0.4)submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Evaluation

In [ ]:
df.head()

,image_id,class_id,x_min,y_min,x_max,y_max,class_name,x_mid,y_mid,w,h,area,width,height
0,9a5094b2563a1ef3ff50dc5c7ff71345,3,339.7,593.0,816.0,787.3,Cardiomegaly,0.564307,0.673975,0.465137,0.189746,92545.09,2080,2336
1,9a5094b2563a1ef3ff50dc5c7ff71345,10,880.0,757.0,923.0,873.0,Pleural effusion,0.880371,0.795898,0.041992,0.113281,4988.00,2080,2336
2,9a5094b2563a1ef3ff50dc5c7ff71345,11,880.0,757.0,923.0,873.0,Pleural thickening,0.880371,0.795898,0.041992,0.113281,4988.00,2080,2336
3,9a5094b2563a1ef3ff50dc5c7ff71345,0,517.0,313.0,639.0,423.0,Aortic enlargement,0.564453,0.359375,0.119141,0.107422,13420.00,2080,2336
4,051132a778e61a86eb147c7c6f564dfe,3,423.7,463.7,907.7,594.3,Cardiomegaly,0.650098,0.516602,0.472656,0.127539,63210.40,2304,2880


In [ ]:
import glob
image_ids = []
PredictionStrings = []
classes = []
scores = []
x_min = []
y_min = []
x_max = []
y_max = []
#pd.merge(df, df_folds[df_folds['fold']!=fold], on='image_id')
fold = 4
df_768 = pd.merge(df, df_folds[df_folds['fold']==fold], on='image_id')
df_768['x_min'] = (df_768['x_min']/1024*768).astype(int)
df_768['y_min'] = (df_768['y_min']/1024*768).astype(int) 
df_768['y_max'] = (df_768['y_max']/1024*768).astype(int) 
df_768['x_max'] = (df_768['x_max']/1024*768).astype(int) 
path = f"runs{fold}/detect/"
IMAGE_SIZE = 768
for file_path in glob.glob(path+'exp/labels/*txt'):
    image_id = file_path.split('/')[-1].split('.')[0]
    #w, h = df.loc[df.image_id==image_id,['width', 'height']].values[0]
    w, h = IMAGE_SIZE, IMAGE_SIZE
    f = open(file_path, 'r')
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
    bboxes = list(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1))#.astype(str))
    for i in range(len(bboxes)//6):
        image_ids.append(image_id)
        classes.append(int(bboxes[i*6]))
        scores.append(bboxes[i*6+1])
        x_min.append(int(bboxes[i*6+2]))
        y_min.append(int(bboxes[i*6+3]))
        x_max.append(int(bboxes[i*6+4]))
        y_max.append(int(bboxes[i*6+5]))

In [ ]:
df_pred = pd.DataFrame({'image_id':image_ids,
                        'class_id':classes,
                        'conf':scores,
                        'x_min':x_min,
                        'y_min':y_min,
                        'x_max':x_max,
                        'y_max':y_max
                       })
df_pred = df_pred.sort_values('conf', ascending=False)

In [ ]:

def get_real_annotations(table):
    res = dict()
    ids = table['ImageID'].values.astype(np.str)
    labels = table['LabelName'].values.astype(np.str)
    xmin = table['XMin'].values.astype(np.float32)
    xmax = table['XMax'].values.astype(np.float32)
    ymin = table['YMin'].values.astype(np.float32)
    ymax = table['YMax'].values.astype(np.float32)

    for i in range(len(ids)):
        id = ids[i]
        label = labels[i]
        if id not in res:
            res[id] = dict()
        if label not in res[id]:
            res[id][label] = []
        box = [xmin[i], ymin[i], xmax[i], ymax[i]]
        res[id][label].append(box)

    return res


def get_detections(table):
    res = dict()
    ids = table['ImageID'].values.astype(np.str)
    labels = table['LabelName'].values.astype(np.str)
    scores = table['Conf'].values.astype(np.float32)
    xmin = table['XMin'].values.astype(np.float32)
    xmax = table['XMax'].values.astype(np.float32)
    ymin = table['YMin'].values.astype(np.float32)
    ymax = table['YMax'].values.astype(np.float32)

    for i in range(len(ids)):
        id = ids[i]
        label = labels[i]
        if id not in res:
            res[id] = dict()
        if label not in res[id]:
            res[id][label] = []
        box = [xmin[i], ymin[i], xmax[i], ymax[i], scores[i]]
        res[id][label].append(box)

    return res


def _compute_ap(recall, precision):
    """ Compute the average precision, given the recall and precision curves.
    Code originally from https://github.com/rbgirshick/py-faster-rcnn.
    # Arguments
        recall:    The recall curve (list).
        precision: The precision curve (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.], recall, [1.]))
    mpre = np.concatenate(([0.], precision, [0.]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap


def mean_average_precision_for_boxes(ann, pred, iou_threshold=0.4, exclude_not_in_annotations=False, verbose=True):
    """
    :param ann: path to CSV-file with annotations or numpy array of shape (N, 6)
    :param pred: path to CSV-file with predictions (detections) or numpy array of shape (N, 7)
    :param iou_threshold: IoU between boxes which count as 'match'. Default: 0.5
    :param exclude_not_in_annotations: exclude image IDs which are not exist in annotations. Default: False
    :param verbose: print detailed run info. Default: True
    :return: tuple, where first value is mAP and second values is dict with AP for each class.
    """

    if isinstance(ann, str):
        valid = pd.read_csv(ann)
    else:
        valid = pd.DataFrame(ann, columns=['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax'])

    if isinstance(pred, str):
        preds = pd.read_csv(pred)
    else:
        preds = pd.DataFrame(pred, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

    ann_unique = valid['ImageID'].unique()
    preds_unique = preds['ImageID'].unique()

    if verbose:
        print('Number of files in annotations: {}'.format(len(ann_unique)))
        print('Number of files in predictions: {}'.format(len(preds_unique)))

    # Exclude files not in annotations!
    if exclude_not_in_annotations:
        preds = preds[preds['ImageID'].isin(ann_unique)]
        preds_unique = preds['ImageID'].unique()
        if verbose:
            print('Number of files in detection after reduction: {}'.format(len(preds_unique)))

    unique_classes = valid['LabelName'].unique().astype(np.str)
    if verbose:
        print('Unique classes: {}'.format(len(unique_classes)))

    all_detections = get_detections(preds)
    all_annotations = get_real_annotations(valid)
    if verbose:
        print('Detections length: {}'.format(len(all_detections)))
        print('Annotations length: {}'.format(len(all_annotations)))

    average_precisions = {}
    for zz, label in enumerate(sorted(unique_classes)):

        # Negative class
        if str(label) == 'nan':
            continue

        false_positives = []
        true_positives = []
        scores = []
        num_annotations = 0.0

        for i in range(len(ann_unique)):
            detections = []
            annotations = []
            id = ann_unique[i]
            if id in all_detections:
                if label in all_detections[id]:
                    detections = all_detections[id][label]
            if id in all_annotations:
                if label in all_annotations[id]:
                    annotations = all_annotations[id][label]

            if len(detections) == 0 and len(annotations) == 0:
                continue

            num_annotations += len(annotations)
            detected_annotations = []

            annotations = np.array(annotations, dtype=np.float64)
            for d in detections:
                scores.append(d[4])

                if len(annotations) == 0:
                    false_positives.append(1)
                    true_positives.append(0)
                    continue

                overlaps = compute_overlap(np.expand_dims(np.array(d, dtype=np.float64), axis=0), annotations)
                assigned_annotation = np.argmax(overlaps, axis=1)
                max_overlap = overlaps[0, assigned_annotation]

                if max_overlap >= iou_threshold and assigned_annotation not in detected_annotations:
                    false_positives.append(0)
                    true_positives.append(1)
                    detected_annotations.append(assigned_annotation)
                else:
                    false_positives.append(1)
                    true_positives.append(0)

        if num_annotations == 0:
            average_precisions[label] = 0, 0
            continue

        false_positives = np.array(false_positives)
        true_positives = np.array(true_positives)
        scores = np.array(scores)

        # sort by score
        indices = np.argsort(-scores)
        false_positives = false_positives[indices]
        true_positives = true_positives[indices]

        # compute false positives and true positives
        false_positives = np.cumsum(false_positives)
        true_positives = np.cumsum(true_positives)

        # compute recall and precision
        recall = true_positives / num_annotations
        precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)

        # compute average precision
        average_precision = _compute_ap(recall, precision)
        average_precisions[label] = average_precision, num_annotations, precision, recall
        if verbose:
            s1 = "{:30s} | {:.6f} | {:7d}".format(label, average_precision, int(num_annotations))
            print(s1)

    present_classes = 0
    precision = 0
    for label, (average_precision, num_annotations, _, _) in average_precisions.items():
        if num_annotations > 0:
            present_classes += 1
            precision += average_precision
    mean_ap = precision / present_classes
    if verbose:
        print('mAP: {:.6f}'.format(mean_ap))
    return mean_ap, average_precisions


def compute_overlap(boxes, query_boxes):
    """
    Args
        a: (N, 4) ndarray of float
        b: (K, 4) ndarray of float
    Returns
        overlaps: (N, K) ndarray of overlap between boxes and query_boxes
    """
    N = boxes.shape[0]
    K = query_boxes.shape[0]
    overlaps = np.zeros((N, K), dtype=np.float64)
    for k in range(K):
        box_area = (
            (query_boxes[k, 2] - query_boxes[k, 0]) *
            (query_boxes[k, 3] - query_boxes[k, 1])
        )
        for n in range(N):
            iw = (
                min(boxes[n, 2], query_boxes[k, 2]) -
                max(boxes[n, 0], query_boxes[k, 0])
            )
            if iw > 0:
                ih = (
                    min(boxes[n, 3], query_boxes[k, 3]) -
                    max(boxes[n, 1], query_boxes[k, 1])
                )
                if ih > 0:
                    ua = np.float64(
                        (boxes[n, 2] - boxes[n, 0]) *
                        (boxes[n, 3] - boxes[n, 1]) +
                        box_area - iw * ih
                    )
                    overlaps[n, k] = iw * ih / ua
    return overlaps

In [ ]:
df_pred_thre015 = df_pred[df_pred.conf > 0.15]
df_pred_thre05 = df_pred[df_pred.conf > 0.5]
df_pred_thre005 = df_pred[df_pred.conf > 0.001]
pred_thre015 = df_pred_thre015[['image_id', 'class_id', 'conf','x_min','x_max','y_min','y_max']].values
pred_thre05 = df_pred_thre05[['image_id', 'class_id', 'conf','x_min','x_max','y_min','y_max']].values
pred_thre005 = df_pred_thre005[['image_id', 'class_id', 'conf','x_min','x_max','y_min','y_max']].values

In [ ]:
#768
anno = df_768[['image_id', 'class_id','x_min','x_max','y_min','y_max']].values
mean_ap_thre015, average_precisions_thre015 = mean_average_precision_for_boxes(anno, pred_thre015, verbose=False)
mean_ap_thre05, average_precisions_thre05 = mean_average_precision_for_boxes(anno, pred_thre05, verbose=False)
mean_ap_thre005, average_precisions_thre005 = mean_average_precision_for_boxes(anno, pred_thre005, verbose=False)
print('mAP with threshold 0.15', round(mean_ap_thre015,4))
print('mAP with threshold 0.5 ', round(mean_ap_thre05,4))
print('mAP with threshold 0.001 ', round(mean_ap_thre005,4))

mAP with threshold 0.15 0.3709
mAP with threshold 0.5  0.1621
mAP with threshold 0.001  0.6025
